In [5]:
# %pip install tensorflow==2.6.0
# %pip install coremltools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.1 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 47.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 28.9 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 22.1.0
    Uninstalling attrs-22.1.0:
      Successfully uninstalled attrs-22.1.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import necessary libraries:
# - TensorFlow: Framework for deep learning.
# - MobileNetV2: A pre-trained model for transfer learning.
# - ImageDataGenerator: Generates batches of image data with real-time data augmentation.
# - Other libraries for numerical operations, class balancing, and model conversion.

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2  # Pre-trained MobileNetV2 model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D  # Layers for fine-tuning
from tensorflow.keras.models import Model  # Functional API for model definition
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data augmentation and preprocessing
import os  # For file and directory operations
import numpy as np  # Numerical operations
from sklearn.utils.class_weight import compute_class_weight  # Compute class weights for imbalanced data
import coremltools as ct  # For CoreML model conversion

# Define directories for the dataset:
# - `train_dir`: Training data directory.
# - `val_dir`: Validation data directory.
# - `test_dir`: Testing data directory.
# - `model_save_path`: Path to save the trained model in H5 format.
# - `mlmodel_save_path`: Path to save the converted CoreML model.

train_dir = '/Users/mtsenk/Downloads/train'  # Path to training data
val_dir = '/Users/mtsenk/Downloads/validation'  # Path to validation data
test_dir = '/Users/mtsenk/Downloads/test'  # Path to testing data
model_save_path = '/Users/mtsenk/Downloads/some/trained_model.h5'  # Path to save model (H5 format)
mlmodel_save_path = '/Users/mtsenk/Downloads/some/trained_model.mlmodel'  # Path to save CoreML model

# Define hyperparameters for the model:
# - `img_size`: Dimensions of input images (224x224 for MobileNetV2).
# - `batch_size`: Number of images per training batch.
# - `epochs`: Total number of training iterations over the dataset.
# - `learning_rate`: Step size for weight updates during training.

img_size = 224  # Image size for resizing
batch_size = 32  # Number of images per batch
epochs = 95  # Number of training epochs
learning_rate = 0.0001  # Initial learning rate

# Create an ImageDataGenerator instance for training:
# - `rescale`: Normalize pixel values to range [0, 1].
# - Other parameters: Augmentations like rotation, shifting, zooming, flipping, etc.

train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=30,  # Randomly rotate images by up to 30 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Apply shearing transformations
    zoom_range=0.2,  # Randomly zoom into images
    horizontal_flip=True,  # Flip images horizontally
    fill_mode="nearest"  # Fill empty pixels with the nearest pixel value
)

# Create an ImageDataGenerator instance for validation and testing:
# - Only normalization is applied to avoid modifying validation/test data.

val_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values for validation
test_datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values for testing

# Create data generators for training, validation, and testing:
# - `flow_from_directory`: Reads images from the specified directory and applies transformations.
# - `target_size`: Resizes images to (img_size, img_size).
# - `batch_size`: Number of images per batch.
# - `class_mode`: Indicates categorical classification.

train_generator = train_datagen.flow_from_directory(
    train_dir,  # Training data directory
    target_size=(img_size, img_size),  # Resize images
    batch_size=batch_size,  # Batch size
    class_mode='categorical'  # Categorical classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,  # Validation data directory
    target_size=(img_size, img_size),  # Resize images
    batch_size=batch_size,  # Batch size
    class_mode='categorical'  # Categorical classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,  # Testing data directory
    target_size=(img_size, img_size),  # Resize images
    batch_size=batch_size,  # Batch size
    class_mode='categorical'  # Categorical classification
)


Found 3536 images belonging to 14 classes.
Found 1177 images belonging to 14 classes.
Found 1184 images belonging to 14 classes.


In [ ]:
# we use tensorflow pre-trained model
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False

Metal device set to: Apple M1 Pro
9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
# Add additional layers on top of the pre-trained base model:
# - `base_model.output`: Output from the pre-trained base model.
# - `GlobalAveragePooling2D`: Reduces the feature map size to a single vector by averaging, which helps prevent overfitting.
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Apply global average pooling to the output of the base model

# Add a fully connected (dense) layer with 128 units and ReLU activation for learning complex features.
x = Dense(128, activation='relu')(x)  # Dense layer with ReLU activation

# Add a dropout layer to randomly set 50% of the neurons to zero during training, reducing overfitting.
x = Dropout(0.5)(x)  # Dropout layer to reduce overfitting

# Add the final output layer:
# - The number of neurons is equal to the number of classes (`train_generator.num_classes`).
# - The softmax activation function is used for multi-class classification.
output = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer for classification

# Define the final model by connecting the input of the base model to the output layer.
model = Model(inputs=base_model.input, outputs=output)  # Create the complete model

# Compile the model to specify:
# - Optimizer: Adam optimizer with a predefined learning rate.
# - Loss function: Categorical cross-entropy for multi-class classification.
# - Evaluation metric: Accuracy to measure the performance of the model.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),  # Adam optimizer with a specified learning rate
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    metrics=['accuracy']  # Metric to evaluate during training and validation
)


In [ ]:
# Train the model using the training data generator:
# - `train_generator`: Provides batches of training images and labels.
# - `epochs`: Number of complete passes through the training dataset.
# - `validation_data`: Data generator for validation to monitor model performance.
history = model.fit(
    train_generator,  # Training data generator
    epochs=epochs,  # Number of epochs
    validation_data=val_generator  # Validation data generator
)

# Save the trained model to the specified file path:
# - Saves the model in H5 format, which includes the model architecture, weights, and optimizer state.
model.save(model_save_path)  # Save the trained model to the specified path
print(f"Model saved to {model_save_path}")  # Print confirmation of saved model


Epoch 1/95
 61/111 [===============>..............] - ETA: 39s - loss: 2.8069 - accuracy: 0.1112

/Users/mtsenk/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


111/111 [==============================] - 120s 1s/step - loss: 2.6280 - accuracy: 0.1581 - val_loss: 2.2029 - val_accuracy: 0.3475
Epoch 2/95
111/111 [==============================] - 118s 1s/step - loss: 2.1262 - accuracy: 0.3111 - val_loss: 1.8017 - val_accuracy: 0.4894
Epoch 3/95
111/111 [==============================] - 120s 1s/step - loss: 1.8108 - accuracy: 0.4219 - val_loss: 1.5069 - val_accuracy: 0.5820
Epoch 4/95
111/111 [==============================] - 118s 1s/step - loss: 1.5450 - accuracy: 0.5040 - val_loss: 1.2908 - val_accuracy: 0.6296
Epoch 5/95
111/111 [==============================] - 121s 1s/step - loss: 1.3667 - accuracy: 0.5625 - val_loss: 1.1534 - val_accuracy: 0.6703
Epoch 6/95
111/111 [==============================] - 121s 1s/step - loss: 1.2720 - accuracy: 0.5902 - val_loss: 1.0736 - val_accuracy: 0.6907
Epoch 7/95
111/111 [==============================] - 113s 1s/step - loss: 1.1632 - accuracy: 0.6380 - val_loss: 0.9856 - val_accuracy: 0.7052
Epoch 8/95

In [5]:
# Fine-tuning
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate / 10),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    epochs=epochs + 5,
    initial_epoch=history.epoch[-1],
    validation_data=val_generator
)


Epoch 95/100
111/111 [==============================] - 135s 1s/step - loss: 0.9597 - accuracy: 0.6937 - val_loss: 0.4528 - val_accuracy: 0.8437
Epoch 96/100
111/111 [==============================] - 121s 1s/step - loss: 0.7176 - accuracy: 0.7653 - val_loss: 0.4481 - val_accuracy: 0.8530
Epoch 97/100
111/111 [==============================] - 120s 1s/step - loss: 0.6282 - accuracy: 0.7921 - val_loss: 0.4356 - val_accuracy: 0.8590
Epoch 98/100
111/111 [==============================] - 119s 1s/step - loss: 0.5504 - accuracy: 0.8190 - val_loss: 0.4285 - val_accuracy: 0.8666
Epoch 99/100
111/111 [==============================] - 120s 1s/step - loss: 0.5200 - accuracy: 0.8224 - val_loss: 0.4143 - val_accuracy: 0.8632
Epoch 100/100
111/111 [==============================] - 121s 1s/step - loss: 0.4779 - accuracy: 0.8380 - val_loss: 0.4085 - val_accuracy: 0.8658


In [ ]:

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")


37/37 [==============================] - 25s 658ms/step - loss: 0.4294 - accuracy: 0.8505
Test Accuracy: 0.8505067825317383


In [ ]:
from sklearn.metrics import precision_score  # Import function to compute precision metric

# Predict class probabilities for the test data using the trained model:
# - `test_generator`: Provides test data in batches.
# - Returns probabilities for each class.
y_pred_probs = model.predict(test_generator)

# Convert predicted probabilities to class indices:
# - `np.argmax`: Gets the index of the highest probability for each prediction.
y_pred = np.argmax(y_pred_probs, axis=1)

# Get true class labels from the test data:
# - `test_generator.classes`: Returns true labels for the test data.
y_true = test_generator.classes

# Calculate the precision score:
# - Compares predicted classes (`y_pred`) with true classes (`y_true`).
# - `average='micro'`: Calculates precision globally by counting total true positives and false positives.
precision = precision_score(y_true, y_pred, average='micro')

# Print the calculated precision:
print(f"Precision: {precision}")  # Displays the precision metric


18/37 [=============>................] - ETA: 12s

/Users/mtsenk/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


37/37 [==============================] - 25s 663ms/step
Precision: 0.07601351351351351


In [ ]:
# converting to ml model for usage in swift app
mlmodel = ct.convert(
    model,
    inputs=[ct.ImageType(shape=(1, img_size, img_size, 3), scale=1.0/255)],
    minimum_deployment_target=ct.target.iOS15
)
mlmodel.save(mlmodel_save_path)
print(f"Core ML model saved to {mlmodel_save_path}")

Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 230.03 passes/s]


Core ML model saved to /Users/mtsenk/Downloads/some/trained_model.mlpackage
